In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Iterable, Tuple, Dict, Any
from trak import TRAKer
from trak.modelout_functions import AbstractModelOutput
import numpy as np
import numpy as np
import torch

from transformers import MusicgenForConditionalGeneration, AutoProcessor

from src.tcav.concept import create_experimental_set
from src.data.text_conditioning import TextConditioning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

random_state = 42
np.random.seed(random_state)
torch.manual_seed(random_state)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(random_state)
# torch.set_grad_enabled(True)
torch.set_grad_enabled(False)

from src.tcav.model import MusicGenCodebooks 

/home/bruno/miniconda3/envs/music-gen-interpretability/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
model.to(device)
model = model.half()  # Convert model weights to float16
model.eval()

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [3]:
custom_model = MusicGenCodebooks(model=model, processor=processor)

/home/bruno/miniconda3/envs/music-gen-interpretability/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [4]:
inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)
display(inputs)

{'input_ids': tensor([[ 2775,     7,  2783,  1463,    28,  7981,    63,  5253,     7,    11,
         13353,     1,     0],
        [ 2777,     7,  2480,  2324,    28,  8002,  5507,     7,    11,  2437,
          5253,     7,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [5]:
custom_model.model.generation_config.return_dict_in_generate = True
custom_model.model.generation_config.output_logits = True

In [6]:
codebooks, logits = custom_model.generate(**inputs, max_new_tokens=255)
display(codebooks)
display(logits)

`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


tensor([[2048,  993, 1132,  ...,  924, 1064,  938],
        [2048,  234, 1938,  ...,  583,  229, 1638],
        [2048, 1668,  314,  ..., 1807,  914,  101],
        ...,
        [2048, 1146,  655,  ..., 1593,  308, 1745],
        [2048,  733,  104,  ..., 1027, 1024,  584],
        [2048, 2047,  953,  ...,  971,  624, 1053]], device='cuda:0')

(tensor([[-0.7759, -3.0508, -4.6445,  ...,  0.1844, -3.5078, -4.8672],
         [-0.0875, -3.3281, -0.7954,  ..., -1.6025,  0.6113,  0.7710],
         [-1.8633,  0.7222, -3.1523,  ..., -1.4365, -1.1973, -3.3516],
         ...,
         [-0.2356, -3.1934, -1.1670,  ..., -1.4893,  0.2942,  1.1328],
         [-1.7178,  0.9385, -2.7930,  ..., -1.1475, -1.1973, -3.1953],
         [ 2.2852,  0.6553,  2.7129,  ..., -0.4873, -1.1924,  2.5508]],
        device='cuda:0'),
 tensor([[ -1.5918,  -3.4121,  -3.6855,  ...,   2.0371,  -3.0469,  -5.8477],
         [ -1.8672, -12.0156,  -1.4990,  ...,  -0.5137,  -2.1250,   1.1982],
         [ -0.9087,  -0.8149,  -0.2042,  ...,  -0.7158,  -3.3691,  -9.0234],
         ...,
         [ -0.4058, -10.1406,   1.3408,  ...,  -0.0632,   0.7490,  -0.1470],
         [  0.9009,   0.6079,   0.9116,  ...,  -0.8628,   0.9619,  -6.1875],
         [ -1.4941,   2.9473,   0.9600,  ...,   0.0200,   0.2957,   1.9189]],
        device='cuda:0'),
 tensor([[ -2.9082,  -4.6367, 

In [8]:
print(codebooks.shape)
print(len(logits))
print(logits[0].shape)

torch.Size([8, 256])
255
torch.Size([16, 2048])


In [ ]:
class MusicGenModelOutput(AbstractModelOutput):
    """
    Custom TRAK task implementation for MusicGen attribution analysis.
    
    This implements the model output function and gradient computation
    specifically for MusicGen's multi-codebook token generation.
    """
    
    def __init__(self, temperature: float = 1.0):
        """
        Initialize MusicGen model output function.
        
        Args:
            temperature: Temperature parameter for softmax computation
        """
        self.temperature = temperature
        self.softmax = nn.Softmax(dim=-1)
        
    def get_output(
        self,
        model,
        weights: Iterable[Tensor],
        buffers: Iterable[Tensor],
        input_ids: Tensor,
        attention_mask: Tensor,
        labels: Tensor,
    ) -> Tensor:
        """
        Compute the model output function for MusicGen.
        
        This implements f(z; θ) = log(p(z; θ) / (1 - p(z; θ)))
        summed over all codebook predictions for multi-stream generation.
        
        Args:
            model: The MusicGen model
            weights: Model parameters
            buffers: Model buffers
            input_ids: Input token IDs
            attention_mask: Attention mask
            labels: Target labels/tokens
            
        Returns:
            Model output value (scalar)
        """
        # Prepare inputs for functional call
        kw_inputs = {
            "input_ids": input_ids.unsqueeze(0) if input_ids.dim() == 1 else input_ids,
            "attention_mask": attention_mask.unsqueeze(0) if attention_mask.dim() == 1 else attention_mask,
        }
        
        # Get model logits using functional call
        logits = torch.func.functional_call(
            model, (weights, buffers), args=(), kwargs=kw_inputs
        )
        
        # Handle multi-dimensional logits for multi-codebook prediction
        if logits.dim() == 4:  # [batch, seq_len, num_codebooks, vocab_size]
            batch_size, seq_len, num_codebooks, vocab_size = logits.shape
            
            # Flatten for easier processing
            logits_flat = logits.view(-1, vocab_size)  # [batch*seq*codebooks, vocab_size]
            labels_flat = labels.view(-1)  # [batch*seq*codebooks]
            
            # Compute correct class probabilities
            probs = F.softmax(logits_flat / self.temperature, dim=-1)
            correct_probs = probs[torch.arange(logits_flat.size(0)), labels_flat]
            
            # Compute model output function: f(z) = log(p / (1-p))
            # Clamp probabilities to avoid numerical issues
            correct_probs = torch.clamp(correct_probs, min=1e-8, max=1-1e-8)
            output_values = torch.log(correct_probs / (1 - correct_probs))
            
            # Sum over all positions and codebooks
            return output_values.sum()
            
        elif logits.dim() == 3:  # [batch, seq_len, vocab_size]
            batch_size, seq_len, vocab_size = logits.shape
            
            # Get batch indices
            batch_indices = torch.arange(batch_size).to(logits.device)
            
            # For sequence generation, we typically predict the next token
            # Take the last position or sum over sequence
            if labels.dim() == 1:  # Single target per sequence
                logits_correct = logits[batch_indices, -1, labels]  # Last position
                
                # Clone logits and mask correct class for margin computation
                cloned_logits = logits[:, -1, :].clone()
                cloned_logits[batch_indices, labels] = torch.tensor(
                    -torch.inf, device=logits.device, dtype=logits.dtype
                )
                
                # Compute margins
                margins = logits_correct - cloned_logits.logsumexp(dim=-1)
                return margins.sum()
            else:
                # Multiple targets (sequence of tokens)
                margins_list = []
                for seq_pos in range(min(seq_len, labels.size(1))):
                    pos_logits = logits[:, seq_pos, :]
                    pos_labels = labels[:, seq_pos]
                    
                    logits_correct = pos_logits[batch_indices, pos_labels]
                    
                    cloned_logits = pos_logits.clone()
                    cloned_logits[batch_indices, pos_labels] = torch.tensor(
                        -torch.inf, device=logits.device, dtype=logits.dtype
                    )
                    
                    margins = logits_correct - cloned_logits.logsumexp(dim=-1)
                    margins_list.append(margins)
                
                return torch.stack(margins_list).sum()
        
        else:
            raise ValueError(f"Unexpected logits shape: {logits.shape}")
    
    def get_out_to_loss_grad(
        self,
        model,
        weights,
        buffers,
        batch: Iterable[Tensor]
    ) -> Tensor:
        """
        Compute the gradient of loss w.r.t. model output.
        
        For the cross-entropy loss, this is: ∂ℓ/∂f = -(1 - p(z; θ))
        
        Args:
            model: The MusicGen model
            weights: Model parameters
            buffers: Model buffers
            batch: Batch of input data
            
        Returns:
            Gradient values for the batch
        """
        input_ids, attention_mask, labels = batch[:3]
        
        # Prepare inputs
        kw_inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        }
        
        # Get model logits
        logits = torch.func.functional_call(
            model, (weights, buffers), args=(), kwargs=kw_inputs
        )
        
        # Compute gradients based on logits dimensionality
        if logits.dim() == 4:  # Multi-codebook case
            batch_size, seq_len, num_codebooks, vocab_size = logits.shape
            
            # Flatten for processing
            logits_flat = logits.view(-1, vocab_size)
            labels_flat = labels.view(-1)
            
            # Compute softmax probabilities
            probs = self.softmax(logits_flat / self.temperature)
            correct_probs = probs[torch.arange(logits_flat.size(0)), labels_flat]
            
            # Gradient: -(1 - p)
            grads = -(1 - correct_probs)
            
            # Reshape back to original batch structure
            grads = grads.view(batch_size, seq_len, num_codebooks)
            
            # Sum over sequence and codebook dimensions for each batch item
            return grads.sum(dim=(1, 2)).clone().detach().unsqueeze(-1)
            
        elif logits.dim() == 3:  # Standard sequence case
            batch_size, seq_len, vocab_size = logits.shape
            
            if labels.dim() == 1:  # Single target per sequence
                # Use last position logits
                last_logits = logits[:, -1, :]
                probs = self.softmax(last_logits / self.temperature)
                correct_probs = probs[torch.arange(batch_size), labels]
                
            else:  # Multiple targets
                # Average over sequence positions
                probs_list = []
                for seq_pos in range(min(seq_len, labels.size(1))):
                    pos_logits = logits[:, seq_pos, :]
                    pos_labels = labels[:, seq_pos]
                    
                    probs = self.softmax(pos_logits / self.temperature)
                    correct_probs = probs[torch.arange(batch_size), pos_labels]
                    probs_list.append(correct_probs)
                
                correct_probs = torch.stack(probs_list).mean(dim=0)
            
            # Gradient: -(1 - p)
            grads = -(1 - correct_probs)
            return grads.clone().detach().unsqueeze(-1)
        
        else:
            raise ValueError(f"Unexpected logits shape: {logits.shape}")

In [ ]:
# Initialize TRAKer with custom MusicGen task
traker = TRAKer(
    model=model,
    task=MusicGenModelOutput,  # Use our custom task implementation
    train_set_size=train_set_size,
    save_dir=save_dir,
    device=device,
    proj_dim=proj_dim,
)

def process_batch(batch):
    """
    Process batch to extract required components for TRAK.
    Adapt this function based on your data loader format.
    """
    if isinstance(batch, dict):
        # If batch is a dictionary (common with HuggingFace datasets)
        return (
            batch['input_ids'],
            batch['attention_mask'], 
            batch['labels']
        )
    else:
        # If batch is already a tuple/list
        return batch[:3]  # Take first 3 elements

# Load model checkpoint
print("Loading model checkpoint...")
traker.load_checkpoint(model.state_dict(), model_id=0)

# Featurize training data
print("Featurizing training data...")
for batch in tqdm(train_loader, desc='Featurizing training data'):
    # Process batch into compatible form
    processed_batch = process_batch(batch)
    processed_batch = [x.to(device) for x in processed_batch]
    
    # Featurize the batch
    traker.featurize(
        batch=processed_batch,
        num_samples=processed_batch[0].shape[0]
    )

# Finalize features
print("Finalizing features...")
traker.finalize_features()

# Start scoring process
print("Starting scoring process...")
traker.start_scoring_checkpoint(
    exp_name='musicgen_attribution',
    checkpoint=model.state_dict(),
    model_id=0,
    num_targets=val_set_size
)

# Score validation data
print("Scoring validation data...")
for batch in tqdm(val_loader, desc='Scoring validation data'):
    processed_batch = process_batch(batch)
    processed_batch = [x.to(device) for x in processed_batch]
    
    traker.score(
        batch=processed_batch,
        num_samples=processed_batch[0].shape[0]
    )

# Finalize and get scores
print("Finalizing scores...")
scores = traker.finalize_scores(exp_name='musicgen_attribution')